In [1]:
import pandas as pd

In [2]:
# Reference to the data source
purchase_data_file = "resources/purchase_data.csv"
purchase_data_df = pd.read_csv(purchase_data_file)

In [3]:
# Calculate total players
total_players = purchase_data_df["SN"].unique().size
# Create DataFrame for Total Players
total_players_df = pd.DataFrame([{"Total Players":total_players}])

In [4]:
# Display DataFrame
total_players_df

,Total Players
0,576


In [5]:
# Calculate data for DataFrame columns
number_unique_items = purchase_data_df["Item ID"].unique().size
average_price = purchase_data_df["Price"].mean()
number_purchases = purchase_data_df["Purchase ID"].unique().size
total_revenue = purchase_data_df["Price"].sum()

# Create DataFrame for Purchasing Analysis 
purchasing_analysis_df = pd.DataFrame([{"Number Of Unique Items":number_unique_items,\
                                        "Average Price":average_price,\
                                        "Number Of Purchases":number_purchases,\
                                        "Total Revenue":total_revenue}])

# Format currency columns
purchasing_analysis_df["Average Price"] = purchasing_analysis_df["Average Price"].\
                                                        map("${:,.2f}".format)
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].\
                                                        map("${:,.2f}".format)

# Assign desired order to the columns
purchasing_analysis_df = purchasing_analysis_df[["Number Of Unique Items",\
                                                 "Average Price",\
                                                 "Number Of Purchases",\
                                                 "Total Revenue"]]

In [6]:
# Display DataFrame
purchasing_analysis_df

,Number Of Unique Items,Average Price,Number Of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [7]:
# Create a DataFrame using group by 
gender_demographics_df = purchase_data_df.groupby("Gender", sort=False).nunique().\
                                            sort_values("SN",ascending=False).\
                                            rename_axis(None).\
                                            rename(columns={"SN":"Total Players"})

In [8]:
# Calculate and format Percent of Players column
gender_demographics_df["Percent of Players"] = (gender_demographics_df["Total Players"] / \
                                                total_players) * 100
gender_demographics_df["Percent of Players"] = gender_demographics_df["Percent of Players"].\
                                                                            map("{:,.2f}".format)

In [9]:
# Select desired columns
gender_demographics_df = gender_demographics_df[["Total Players","Percent of Players"]]

In [10]:
# Display DataFrame
gender_demographics_df

,Total Players,Percent of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [11]:
# Create DataFrame grouped by Gender using the agg method
gender_purchase_df = purchase_data_df.groupby(["Gender"])["SN"].\
                                    agg([("Purchase Count","count")]).join(\
                     purchase_data_df.groupby(["Gender"])["Price"].\
                                    agg([("Average Purchase Price","mean"),\
                                    ("Total Purchase Value","sum")]))
gender_purchase_df["Avg Total Purchase per Person"] = gender_purchase_df["Total Purchase Value"] / \
                                                purchase_data_df.groupby(["Gender"])["SN"].nunique()

In [12]:
# Format currency columns
gender_purchase_df["Average Purchase Price"] = gender_purchase_df["Average Purchase Price"].\
                                                                        map("${:,.2f}".format)
gender_purchase_df["Total Purchase Value"] = gender_purchase_df["Total Purchase Value"].\
                                                                        map("${:,.2f}".format)
gender_purchase_df["Avg Total Purchase per Person"] = \
                                            gender_purchase_df["Avg Total Purchase per Person"].\
                                                                        map("${:,.2f}".format)

In [13]:
# Display DataFrame
gender_purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [14]:
# Create bins and their lables
age_range_value = [0, 9, 14, 19, 24, 29, 34, 39, 150]
age_range_label = ["<10", "10-14","15-19","20-24","25-29","30-34","35-40","40+"]

In [15]:
# Create DataFrame using GroupBy 
age_purchase_df = pd.DataFrame(purchase_data_df.groupby(["Age"])["SN"].nunique().reset_index())

In [16]:
# Add the bin lables to the DataFrame
age_purchase_df["Age Range"] = pd.cut(age_purchase_df["Age"],age_range_value,labels=age_range_label)

In [17]:
# Calculate the sum of Ages by bin label
age_purchase_df = age_purchase_df.groupby(["Age Range"])["SN"].\
                                    agg([("Total Count","sum")]).\
                                    rename_axis(None)

In [18]:
# Add Percentage of Players to the DataFrame and format it
age_purchase_df["Percentage of Players"] = ((age_purchase_df["Total Count"]/total_players)*100).\
                                                map("{:,.2f}".format)

In [19]:
# Display the DataFrame
age_purchase_df

,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-40,31,5.38
40+,12,2.08


In [20]:
# Add column Age Range using bins / bin labels
purchase_data_df["Age Range"] = pd.cut(purchase_data_df["Age"],age_range_value,labels=age_range_label)

# Create DataFrame by using GroupBy 
age_purchase_df = purchase_data_df.groupby(["Age Range"])["SN"].\
                        agg([("Purchase Count","count")]).join(\
                   purchase_data_df.groupby(["Age Range"])["Price"].\
                        agg([("Average Purchase Price","mean"),\
                             ("Total Purchase Value","sum")]))
# Add column for avg. purchase per person
age_purchase_df["Avg Total Purchase per Person"] = age_purchase_df["Total Purchase Value"] / \
                                        purchase_data_df.groupby(["Age Range"])["SN"].nunique()

In [21]:
# Reset the index so Age Range becomes a column again
age_purchase_df = age_purchase_df.reset_index()

In [22]:
# Use a sort map to assign the Age Range column arbitrary sort order
sort_order = {"10-14":0,"15-19":1,"20-24":2,"25-29":3,"30-34":4,"35-40":5,"40+":6, "<10":7}
# Add the sort_order map as a colum to the DataFrame mapped to the Age Range
age_purchase_df["Sort Order"] = age_purchase_df["Age Range"].map(sort_order)
# Sort the values using the new column
age_purchase_df.sort_values("Sort Order",inplace=True)
# Set Age Range as the DataFrame index
age_purchase_df.set_index("Age Range", inplace=True)

In [23]:
# Create a new DataFrame with the desired output columns as well as removing the index name column
age_purchase_df = age_purchase_df[["Purchase Count",\
                                   "Average Purchase Price",\
                                   "Total Purchase Value",\
                                   "Avg Total Purchase per Person"]].\
                                    rename_axis(None)                                  

In [24]:
# Format currency columns
age_purchase_df["Average Purchase Price"] = age_purchase_df["Average Purchase Price"].\
                                                            map("${:,.2f}".format)
age_purchase_df["Total Purchase Value"] = age_purchase_df["Total Purchase Value"].\
                                                            map("${:,.2f}".format)
age_purchase_df["Avg Total Purchase per Person"] = age_purchase_df["Avg Total Purchase per Person"].\
                                                            map("${:,.2f}".format)

In [25]:
# Display the DataFrame
age_purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-40,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
<10,23,$3.35,$77.13,$4.54


In [26]:
# Create 2 DataFrames using the GroupBy 
top_spenders_df = purchase_data_df.groupby("SN")["Item ID"].agg([("Purchase Count","count")]).\
                  join(\
                  purchase_data_df.groupby("SN")["Price"].agg([("Average Purchase Price","mean"),\
                                                               ("Total Purchase Value","sum")]))
# Sort the DataFrame in descending order using the Total Purchase Value column
top_spenders_df.sort_values("Total Purchase Value", inplace=True,ascending = False)

In [27]:
# Format currency columns
top_spenders_df["Average Purchase Price"] = top_spenders_df["Average Purchase Price"].\
                                                            map("${:,.2f}".format)
top_spenders_df["Total Purchase Value"] = top_spenders_df["Total Purchase Value"].\
                                                            map("${:,.2f}".format)

In [28]:
# Display top 5 rows from DataFrame
top_spenders_df.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [29]:
# Create 2 DataFrames by using the GroupBy and .agg method 
most_popular_df = purchase_data_df.groupby(["Item ID","Item Name"])["Purchase ID"].\
                            agg([("Purchase Count","count")]).join(\
                  purchase_data_df.groupby(["Item ID", "Item Name"])["Price"].\
                            agg([("Item Price","min"),\
                                 ("Total Purchase Value","sum")]))

In [30]:
# Display top 5 rows from DataFrame 
# Sort the DataFrame in descending order using the Purchase Count column
pd.options.display.float_format = '${:,.2f}'.format
most_popular_df.sort_values("Purchase Count", ascending = False, inplace=False).head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [31]:
most_popular_df.sort_values("Total Purchase Value", ascending = False, inplace=False).head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
